In [ ]:
#    This file is part of DEAP.
#    This implements the NSGA-II in an easy way because it makes us of DEAP subroutines
#    The non dominated sort and crowding distance are solved by a simiple call to DEAP subroutines
#    and their implementation is hidden.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import array
import random
import json
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv
import numpy

from math import sqrt
from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools
import matplotlib.pyplot as plt

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
torchvision.__version__

#normalise pixel values (range:-1 to 1, mean:0.5, s.d:0.5)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size=10000

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

totalImages   = len(trainloader.dataset)
miniAmount    = 1000
numOfLoaders  = totalImages // miniAmount

print("Number of loaders to create: " + str(numOfLoaders)) # 10 Loaders

miniLoaders = []

for i in range(numOfLoaders):
  startIdx        = i * miniAmount
  endIdx          = (i + 1) * miniAmount if i < numOfLoaders - 1 else totalImages
  subset          = torch.utils.data.Subset(trainset, range(startIdx, endIdx))  # Creates a subset of the whole training set
  SubTrainLoader  = torch.utils.data.DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=2)

  miniLoaders.append(SubTrainLoader)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 52),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(52, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

PopModel=Net()
PopModel.to(device)

PATH = ('./40_epoch_32_batch_SGD_net.pth')
PopModel = Net()
PopModel.load_state_dict(torch.load(PATH))
final_layer = PopModel.classifier[-1]

criterion = nn.CrossEntropyLoss()
nn.init.xavier_uniform(final_layer.weight)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# creator.create("Individual", list, fitness=creator.FitnessMulti)
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMax)
NDIM = 530
toolbox = base.Toolbox()

BOUND_LOW, BOUND_UP = -1.0, 1.0

def uniform(low, up, size=None):
    try:
        return [random.uniform(a, b) for a, b in zip(low, up)]
    except TypeError:
        return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]



def calcFitness(individual, miniNumber):
    weights = numpy.square(numpy.array(individual))
    particleweightsNP1 = numpy.array(individual)
    particleweightsNP = particleweightsNP1[:520]
    biases = numpy.array(particleweightsNP1[-10:])
    biases = torch.from_numpy(biases).float()
    final_layer.bias = torch.nn.Parameter(biases.float())

    #converting to the correct shape
    reshapedWeights = particleweightsNP.reshape(10,52)

    #converting to torch array
    torchWeights = torch.from_numpy(reshapedWeights).float()

    #setting the weights of the final layer to these weights
    final_layer.weight = torch.nn.Parameter(torchWeights.float())

    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

#     print("Calc for Individual")
    with torch.no_grad():
        for data in miniLoaders[miniNumber]:
            inputs, labels = data[0].to(device), data[1].to(device)
            PopModel.to(device)
            outputs = PopModel(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = 100*(correct_predictions / total_samples)
#     print(f"For entire training set, total samples: {total_samples}, correct predictions: {correct_predictions}, accuracy: {accuracy}")

    return accuracy,

toolbox.register("attr_float", uniform, BOUND_LOW, BOUND_UP, NDIM)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", calcFitness)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0, indpb=1.0/NDIM)
toolbox.register("select", tools.selRoulette, fit_attr='fitness')


def main(seed=None):
    random.seed(seed)

    NGEN = 100
    MU =  100
    CXPB = 0.8

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "min", "max"

    pop = toolbox.population(n=MU)
    miniCounter = 0

    #evaluating the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    print("Evaluate fitnesses before generation loop..")
    # fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    fitnesses = toolbox.map(lambda part: toolbox.evaluate(part, miniCounter), invalid_ind)

    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    #assigning crowding distance to the individuals
    pop = toolbox.select(pop, len(pop))

    record = stats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print(logbook.stream)

    #beginning the generational process
    for gen in range(1, NGEN):
        #vary the population
        offspring = tools.selTournamentDCD(pop, len(pop))
        offspring = [toolbox.clone(ind) for ind in offspring]

        for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        #making pairs of all (even,odd) in offspring
            if random.random() <= CXPB:
                toolbox.mate(ind1, ind2)

            toolbox.mutate(ind1)
            toolbox.mutate(ind2)
            del ind1.fitness.values, ind2.fitness.values

        #evaluating the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
#         print(f"Evaluate fitnesses at {NGEN} generation..")
        # fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        fitnesses = toolbox.map(lambda part: toolbox.evaluate(part, miniCounter), invalid_ind)

        for ind, fit in zip(invalid_ind, fitnesses):
#             print(f"Fitness for individual is {fit}")
            ind.fitness.values = fit

        #selecting the next generation population
        pop = toolbox.select(pop + offspring, MU)

        #we now have a new evolved population, the logbook can just record the highest accuracy and lowest regulariser for that generation
        #generations continue untill 100
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        print(logbook.stream)

        miniCounter += 1

        if miniCounter >= (len(miniLoaders)):
          miniCounter = 0
          print("Mini Counter reset!")

    print("Final population hypervolume is %f" % hypervolume(pop, [11.0, 11.0]))

    return pop, logbook

if __name__ == "__main__":
    pop, stats = main()